In [1]:
import pandas as pd
import toml
from pathlib import Path

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1f}'.format)


In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
# trip data
trip = pd.read_csv(output_path / 'agg\dash\person_trips.csv')
# person data
person = pd.read_csv(output_path / 'agg\dash\person_geog.csv')
# vmt data
vmt = pd.read_csv(output_path / 'agg/dash/person_vmt.csv')

# list of equity geographies
equity_geogs = summary_config['hh_equity_geogs']
# not_equity_geogs = ["NOT in " + item for item in equity_geogs]

In [4]:
# TRIPS
df_trip = trip.copy()
# add home RGC
df_trip['is_rgc'] = 'Not in RGC'
df_trip.loc[df_trip['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
# add trip type
df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'

# PERSONS
df_person = person.copy()
# add home RGC
df_person['is_rgc'] = 'Not in RGC'
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# VMT
df_vmt = vmt.copy()
# add home RGC
df_vmt['is_rgc'] = 'Not in RGC'
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# Select only walk and bike trips
df_vmt_bp = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])].copy()
# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)].copy()

# # not in equity geography
# df_person[not_equity_geogs] = 1 - df_person[equity_geogs]
# df_trip[not_equity_geogs] = 1 - df_trip[equity_geogs]
# df_vmt[not_equity_geogs] = 1 - df_vmt[equity_geogs]
# df_vmt_bp[not_equity_geogs] = 1 - df_vmt_bp[equity_geogs]

# total population by equity geography
equity_geogs_population = df_person[equity_geogs].apply(lambda x: x * df_person['psexpfac']).sum().reset_index()
equity_geogs_population.columns = ['Equity Group', 'psexpfac']

# total population by "NOT in" equity geography
# not_equity_geogs_population = df_person[not_equity_geogs].apply(lambda x: x * df_person['psexpfac']).sum().reset_index()
# not_equity_geogs_population.columns = ['Equity Group', 'psexpfac']

## Trips per Day by Resident

In [5]:
def trips_per_day(geog, map=False): 
    """
    Calculate trips per day by geography
    """

    # total population by geography
    df1 = df_person.groupby([geog], as_index=False)['psexpfac'].sum().set_index(geog)

    # total trips by geography
    df2 = df_trip.groupby([geog], as_index=False)['trexpfac'].sum().set_index(geog)

    # total trips by trip type and geography
    df3 = df_trip.groupby([geog, 'trip_type'], as_index=False)['trexpfac'].sum().set_index(geog)
    df3 = df3.pivot(columns='trip_type', values='trexpfac')

    # Merge the dataframes
    df = df1.merge(df2, left_index=True, right_index=True)
    df = df.merge(df3, left_index=True, right_index=True)

    if map:
        df.index = df.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

    # regional totals
    df.loc['Region', ['Work','Non-Work','psexpfac','trexpfac']] = df[['Work','Non-Work','psexpfac','trexpfac']].sum()

    df['Total Trips per Day'] = df['trexpfac']/df['psexpfac']
    df['Work Trips per Day'] = df['Work']/df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work']/df['psexpfac']


    
    return df[['Work Trips per Day', 'Non-Work Trips per Day','Total Trips per Day']]

In [6]:
df = trips_per_day('hh_county')
df = df[df.index != 'Outside Region']
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_county,,,
King,0.5,3.3,3.8
Kitsap,0.4,3.3,3.7
Pierce,0.4,3.1,3.6
Snohomish,0.5,3.2,3.7
Region,0.5,3.2,3.7


In [7]:
trips_per_day('is_rgc')

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
is_rgc,,,
In RGC,0.7,3.3,4.0
Not in RGC,0.5,3.2,3.7
Region,0.5,3.2,3.7


In [8]:
trips_per_day('hh_rgc')

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rgc,,,
Auburn,0.4,3.5,3.9
Bellevue,0.7,3.4,4.0
Bothell Canyon Park,0.6,3.3,3.9
Bremerton,0.5,3.5,4.0
Burien,0.6,3.3,3.8
Everett,0.5,3.4,4.0
Federal Way,0.4,3.5,3.9
Greater Downtown Kirkland,0.6,3.4,3.9
Kent,0.5,3.4,3.8


In [9]:
trips_per_day('hh_rg_proposed')

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rg_proposed,,,
Cities and Towns,0.5,3.2,3.7
Core Cities,0.5,3.3,3.8
High Capacity Transit Communities,0.5,3.2,3.7
Metropolitan Cities,0.6,3.3,3.9
Rural Areas,0.4,3.1,3.5
Urban Unincorporated Areas,0.4,3.2,3.6
Region,0.5,3.2,3.7


### Equity Focus Areas

In [10]:
trips_per_day('hh_efa_poc', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_poc,,,
Below Regional Average,0.5,3.2,3.7
Above Regional Average,0.5,3.3,3.8
Higher Share of Equity Population,0.5,3.3,3.8
Region,0.5,3.2,3.7


In [11]:
trips_per_day('hh_efa_pov200', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_pov200,,,
Below Regional Average,0.5,3.3,3.8
Above Regional Average,0.5,3.2,3.7
Higher Share of Equity Population,0.5,3.2,3.7
Region,0.5,3.2,3.7


In [12]:
trips_per_day('hh_efa_lep', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_lep,,,
Below Regional Average,0.5,3.2,3.7
Above Regional Average,0.5,3.3,3.8
Higher Share of Equity Population,0.5,3.3,3.8
Region,0.5,3.2,3.7


In [13]:
trips_per_day('hh_efa_dis', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_dis,,,
Below Regional Average,0.5,3.3,3.8
Above Regional Average,0.5,3.2,3.7
Higher Share of Equity Population,0.4,3.2,3.7
Region,0.5,3.2,3.7


In [14]:
trips_per_day('hh_efa_older', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_older,,,
Below Regional Average,0.5,3.2,3.8
Above Regional Average,0.5,3.2,3.7
Higher Share of Equity Population,0.4,3.2,3.7
Region,0.5,3.2,3.7


In [15]:
trips_per_day('hh_efa_youth', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_youth,,,
Below Regional Average,0.5,3.3,3.8
Above Regional Average,0.5,3.2,3.7
Higher Share of Equity Population,0.4,3.2,3.7
Region,0.5,3.2,3.7


## Miles Driven per Day by Resident

In [16]:
# pd.options.display.float_format = '{:0,.1f}'.format
df_vmt = pd.read_csv(output_path / 'agg/dash/person_vmt.csv')
df_person = person.copy()

df_vmt['is_rgc'] = 'Not in RGC'
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
df_person['is_rgc'] = 'Not in RGC'
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'


# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

In [17]:
def miles_per_day(geog, map=False): 
    """
    Calculate trips per day by geography
    """

    # total population by geography
    df1 = df_person.groupby([geog], as_index=False)['psexpfac'].sum().set_index(geog)

    # total miles by geography
    df2 = df_vmt.groupby([geog], as_index=False)['travdist_wt'].sum().set_index(geog)

    # Merge the dataframes
    df = df1.merge(df2, left_index=True, right_index=True)

    if map:
        df.index = df.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

    # regional totals
    df.loc['Region', ['psexpfac','travdist_wt']] = df[['psexpfac','travdist_wt']].sum()

    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

In [18]:
df = miles_per_day('hh_county')
df = df[df.index != 'Outside Region']
df

,Average Miles per Person
hh_county,
King,13.4
Kitsap,13.8
Pierce,15.1
Snohomish,16.4
Region,14.3


In [19]:
miles_per_day('is_rgc')

,Average Miles per Person
is_rgc,
In RGC,6.2
Not in RGC,14.9
Region,14.3


In [20]:
miles_per_day('hh_rgc')

,Average Miles per Person
hh_rgc,
Auburn,11.0
Bellevue,7.0
Bothell Canyon Park,14.7
Bremerton,7.2
Burien,12.5
Everett,8.8
Federal Way,11.4
Greater Downtown Kirkland,11.8
Kent,10.0


In [21]:
miles_per_day('hh_rg_proposed')

,Average Miles per Person
hh_rg_proposed,
Cities and Towns,17.5
Core Cities,13.9
High Capacity Transit Communities,14.7
Metropolitan Cities,9.9
Rural Areas,22.1
Urban Unincorporated Areas,16.0
Region,14.3


In [22]:
df = pd.DataFrame()
for name, col in {
    "People of Color": "hh_efa_poc",
    "Income": "hh_efa_pov200",
    "LEP": "hh_efa_lep",
    "Disability": "hh_efa_dis",
    "Older Adults": "hh_efa_older",
    "Youth": "hh_efa_youth"
}.items():
    df[name] = miles_per_day(col, map=True)
df

,People of Color,Income,LEP,Disability,Older Adults,Youth
hh_efa_poc,,,,,,
Below Regional Average,15.5,15.3,14.8,14.5,13.6,12.9
Above Regional Average,13.2,13.7,14.1,14.5,15.1,15.7
Higher Share of Equity Population,12.7,11.7,13.1,13.3,15.5,16.2
Region,14.3,14.3,14.3,14.3,14.3,14.3


## Delay

In [23]:
pd.options.display.float_format = '{:0,.1f}'.format

def delay_per_person(geog):

    df_person = pd.read_csv(output_path / 'agg/dash/person_geog.csv')
    df_person['is_rgc'] = 'Not in RGC'
    df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

    df = pd.read_csv(output_path / 'agg/dash/trip_time_total.csv')
    df['is_rgc'] = 'Not in RGC'
    df.loc[df['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
    df = df[(df['mode'].isin(['SOV','HOV2','HOV3+'])&(df['dorp']==1))]
    df = df.groupby(geog).sum()[['travtime_wt']]

    df2 = pd.read_csv(output_path / 'agg/dash/trip_sov_ff_time.csv')
    df2['is_rgc'] = 'Not in RGC'
    df2.loc[df2['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
    df2 = df2[(df2['mode'].isin(['SOV','HOV2','HOV3+'])&(df2['dorp']==1))]
    df2 = df2.groupby(geog).sum()[['sov_ff_time_wt']]
    df = df2.merge(df, on=geog)

    # Hours of delay from travel time (in min)
    df['Total Delay Hours'] = (df['travtime_wt'] - df['sov_ff_time_wt'])/60
    # Set any negative delay to 0
    df.loc[df['Total Delay Hours'] < 0, 'Total Delay Hours'] = 0

    df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = df.merge(df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Minutes of Delay per Person'] = df['Total Delay Hours']/df['psexpfac']*60

    df['Annual Hours of Delay per Person'] = df['Average Minutes of Delay per Person']*summary_config['weekday_to_annual']/60

    df[['Total Delay Hours',
        'Annual Hours of Delay per Person']] = df[['Total Delay Hours', 'Annual Hours of Delay per Person']].astype(int).map('{:,}'.format)


    return df[['Total Delay Hours','Average Minutes of Delay per Person','Annual Hours of Delay per Person']]

df = delay_per_person('hh_county')
df = df[df.index != 'Outside Region']
df

,Total Delay Hours,Average Minutes of Delay per Person,Annual Hours of Delay per Person
hh_county,,,
King,"102,167",2.7,14
Kitsap,"5,010",1.1,5
Pierce,"35,486",2.3,12
Snohomish,"45,780",3.2,17
Region,"188,445",2.6,13


In [24]:
delay_per_person('is_rgc')

,Total Delay Hours,Average Minutes of Delay per Person,Annual Hours of Delay per Person
is_rgc,,,
In RGC,"5,100",1.1,5
Not in RGC,"183,344",2.7,14
Region,"188,445",2.6,13


In [25]:
df = delay_per_person('hh_rgc')

df

,Total Delay Hours,Average Minutes of Delay per Person,Annual Hours of Delay per Person
hh_rgc,,,
Auburn,53,1.4,7
Bellevue,439,1.6,8
Bothell Canyon Park,56,5.0,26
Bremerton,63,1.2,6
Burien,132,2.0,10
Everett,137,1.2,6
Federal Way,12,1.2,6
Greater Downtown Kirkland,443,3.2,17
Kent,69,1.9,10


In [26]:
delay_per_person('hh_rg_proposed')

,Total Delay Hours,Average Minutes of Delay per Person,Annual Hours of Delay per Person
hh_rg_proposed,,,
Cities and Towns,"18,139",2.8,14
Core Cities,"47,321",2.8,15
High Capacity Transit Communities,"50,113",3.1,16
Metropolitan Cities,"40,332",1.9,10
Rural Areas,"23,848",2.6,13
Urban Unincorporated Areas,"8,690",2.9,15
Region,"188,445",2.6,13
